In [70]:
from datetime import datetime , time , timedelta
from __future__ import division
import calendar

def add_months(sourcedate,months):
     month = sourcedate.month - 1 + months
     year = sourcedate.year + month // 12
     month = month % 12 + 1
     day = min(sourcedate.day,calendar.monthrange(year,month)[1])
     return datetime(year,month,day)


def update_general_leaves(date_of_joining , last_updated ,leaves_remaining ,leaves_in_probation, first_year, fiscal_year,
                          probation_period, user , leaves_limit , leaves_availed,
                         probation_leaves_limit , current_time=None):
    
    if current_time == None:
        current_time = datetime.today()
    probation_ending_period = add_months(date_of_joining,probation_period)
    days_in_probation = (probation_ending_period - date_of_joining).days
    

        
    if first_year:

        if date_of_joining == last_updated:
            time_since_last_update = (current_time - probation_ending_period).days
        else:  
            time_since_last_update = (current_time - last_updated).days
        
        
        if (probation_ending_period >= current_time):
            return (leaves_in_probation - leaves_availed)
        else :
            #IS not on probation 
            # Assumption first year rule applies from end of probation to start of new fisca year  
            days_in_year = 366 if calendar.isleap(fiscal_year.year) else 365
            remaining_days = days_in_year - days_in_probation
            leaves_per_day = (leaves_limit-probation_leaves_limit)/ remaining_days
            leaves_remaining = ((time_since_last_update * leaves_per_day) + leaves_remaining) - leaves_availed

    else:
        print('a')
        fiscal_year = fiscal_year.replace(fiscal_year.year-1)
        if fiscal_year == last_updated:
            time_since_last_update = ((current_time - fiscal_year).days)
        else:  
            time_since_last_update = (current_time - last_updated).days
            
        days_in_year = 366 if calendar.isleap(fiscal_year.year) else 365
        leaves_per_day = leaves_limit / days_in_year
        leaves_remaining = (time_since_last_update *leaves_per_day)+ leaves_remaining - leaves_availed
        return leaves_remaining


    last_day_of_fiscal_year = fiscal_year - timedelta(days=1)
    if current_time.day == last_day_of_fiscal_year.day  and  current_time.month == last_day_of_fiscal_year.month:
        leaves_remaining = leaves_remaining + leaves_per_day
        
    return  leaves_remaining



In [78]:
update_general_leaves(
    date_of_joining = datetime(2018,7,1), 
    last_updated = datetime(2020,1,30),
    leaves_remaining = 14.58904109589041,
    leaves_in_probation =5 ,
    first_year = False,
    fiscal_year =  datetime(2020,7,1),
    probation_period = 3,
    user = 1,
    leaves_limit = 25,
    leaves_availed = 0,
    probation_leaves_limit = 5 ,
    current_time=datetime(2020,6,30)
)

a


25.0

In [80]:
from dateutil.relativedelta import relativedelta


In [108]:
today = datetime(2018,6,30)
fiscal_year = datetime(2018,7,1)
perminent_date = datetime(2018,8,1)
sofy = fiscal_year - relativedelta(years=1)
dsofy = abs((today-sofy).days)
dsofe = abs((today - perminent_date).days)
1 if dsofy > dsofe else 0

1

In [169]:
def add_months(sourcedate,months):
     month = sourcedate.month - 1 + months
     year = sourcedate.year + month // 12
     month = month % 12 + 1
     day = min(sourcedate.day,calendar.monthrange(year,month)[1])
     return datetime(year,month,day)

    
def is_first_year(today , fiscal_year , doj, probation_period ):
    if today==None:
        today = datetime.now().date()
    fiscal_year = datetime.strptime(fiscal_year , '%Y-%m-%d')
    today = today.date()
    fiscal_year = datetime(2018,7,1)
    perminent_date = (add_months(doj , probation_period)).date()
    sofy = (fiscal_year - relativedelta(years=1)).date()
    dsofy = abs((today-sofy).days)
    dsofe = abs((today - perminent_date).days)
    return 1 if dsofy > dsofe else 0

is_first_year(today=datetime(2018,7,1) , fiscal_year='2018-07-01' , doj=datetime(2018,5,1) , probation_period=3 )

1